# Springboard--DSC Program

# Capstone Project 1 - Data Wrangling 
### by Ellen A. Savoye

The data collected is from a Kaggle competition, Jigsaw Unintended Bias in Toxicity Classification, via the Kaggle API. Of the 7 files in the zipped data, we will be focusing on the 'train' data. The original 'train' data is comprised of 45 columns containing information on toxicity and identity labels, comments, and metadata. 

### Import packages and data

In [1]:
#!pip install kaggle

In [2]:
import pandas as pd
import numpy as np
import json
import os
from os import path
import shutil
from zipfile import ZipFile

from kaggle.api.kaggle_api_extended import KaggleApi

In [3]:
#Import data from Kaggle API
api = KaggleApi()
api.authenticate()
files = api.competition_download_files("jigsaw-unintended-bias-in-toxicity-classification")

# create conditional to check for zip and csv file already

In [4]:
src = "C:\\Users\\ellen\\Documents\\GitHub\\Data_Science_Career_Track\\Capstone_1\\Code\\"
dst = "C:\\Users\\ellen\\Documents\\GitHub\\Data_Science_Career_Track\\Capstone_1\\Data\\"

# Move the jigsaw zip file to the Data folder
files = [i for i in os.listdir(src) if i.startswith("jigsaw") and path.isfile(path.join(src, i))]
for f in files:
    shutil.move(path.join(src, f), dst)

In [5]:
zipfile_name = 'jigsaw-unintended-bias-in-toxicity-classification.zip'
with ZipFile(dst + zipfile_name, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    print(zipObj.namelist())
    zipObj.extract('train.csv', path = dst)

['identity_individual_annotations.csv', 'sample_submission.csv', 'test.csv', 'test_private_expanded.csv', 'test_public_expanded.csv', 'toxicity_individual_annotations.csv', 'train.csv']


In [6]:
# Read in the train dataset
csv_filename = 'train.csv'
train_data = pd.read_csv(dst + csv_filename, low_memory=False)

# Output the number of rows
print("Total rows: {0}".format(len(train_data)))

# See which headers are available
print(list(train_data))

Total rows: 1804874
['id', 'target', 'comment_text', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation', 'physical_disability', 'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date', 'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree', 'sexual_explicit', 'identity_annotator_count', 'toxicity_annotator_count']


The metadata from Civil Comments platform is contained in the following columns: created_date, publication_id, parent_id, article_id, rating, funny, wow, sad, likes, and disagree.

Given that we're not using Civil Comments' rating or tags, these columns will be removed before checking data types or for missing values.

In [7]:
# Removing Civil Comments metadata columns
slimmed_train_data = train_data.drop(['created_date', 'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree'], axis=1)

In [8]:
#function for counting null records:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
print(slimmed_train_data.apply(num_missing, axis=0))

Missing values per column:
id                                           0
target                                       0
comment_text                                 0
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                           

In [9]:
# Find percent of missing values for each column instead of number of records

percent_missing = slimmed_train_data.isnull().sum() * 100 / len(slimmed_train_data)
print(round(percent_missing,1))

id                                      0.0
target                                  0.0
comment_text                            0.0
severe_toxicity                         0.0
obscene                                 0.0
identity_attack                         0.0
insult                                  0.0
threat                                  0.0
asian                                  77.6
atheist                                77.6
bisexual                               77.6
black                                  77.6
buddhist                               77.6
christian                              77.6
female                                 77.6
heterosexual                           77.6
hindu                                  77.6
homosexual_gay_or_lesbian              77.6
intellectual_or_learning_disability    77.6
jewish                                 77.6
latino                                 77.6
male                                   77.6
muslim                          

In [10]:
# Data type of each column

slimmed_train_data.dtypes

id                                       int64
target                                 float64
comment_text                            object
severe_toxicity                        float64
obscene                                float64
identity_attack                        float64
insult                                 float64
threat                                 float64
asian                                  float64
atheist                                float64
bisexual                               float64
black                                  float64
buddhist                               float64
christian                              float64
female                                 float64
heterosexual                           float64
hindu                                  float64
homosexual_gay_or_lesbian              float64
intellectual_or_learning_disability    float64
jewish                                 float64
latino                                 float64
male         

There are 35 columns in the slimmed down train dataframe. Of those 35, only one column, 'comment_text', is an object. The remaining 34 are either float64 or int64. 'Comment_text' contains the individual comments that we need to analyze.

In [11]:
slimmed_train_data.comment_text.head(5)

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text, dtype: object

In [12]:
# View unique records in a particular column

#sorted(slimmed_train_data.target.unique())
slimmed_train_data.target.unique()

array([0.        , 0.89361702, 0.66666667, ..., 0.87726476, 0.01116838,
       0.87008821])

Toxicity and identity labels range from 0.0-1.0. The value represents the fraction of raters who believed the label fit the comment. Toxicity labels do not have any missing values. According to the competition details, a subset of comments have been labeled with a variety of identity attributes that have been mentioned in the comment. As such, every identity label is missing ~78% of the values per column. The subset comprises approximately 22% of the data. 

Two examples of how labeling works are as follows:
Example 1: 
    - Comment: I'm a white woman in my late 60's and believe me, they are not too crazy about me either!!
    - Toxicity Labels: All 0.0
    - Identity Mention Labels: female: 1.0, white: 1.0 (all others 0.0)

Example 2: 
    - Comment: Continue to stand strong LGBT community. Yes, indeed, you'll overcome and you have.
    - Toxicity Labels: All 0.0
    - Identity Mention Labels: homosexual_gay_or_lesbian: 0.8, bisexual: 0.6, transgender: 0.3 (all others 0.0)


'Target' is the toxicity label. 'Severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', and 'sexual_explicit' are toxicity sub types. All toxicity labels can be converted to categorical variables by using >= 0.5 as a positive indicator (1). 

Aside from 'id', 'comment_text', 'identity_annotator_count' and 'toxicity_annotator_count', the same conversion can be applied to the remaining identity columns. 'Id' is a unique identifier for each comment but may not hold value to keep in the data frame. 'Identity_annotator_count' and 'toxicity_annotator_count' are metadata columns from Jigsaw and may not hold value either. However, I'm not removing them until I do my exploratory data analysis to determine if they offer valuable insights. 

'Comment_text' will need to be cleaned, vectorized, and eventually create a design matrix.  

In [18]:
# Checking for blank string records
np.where(slimmed_train_data.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [20]:
# Checking the range of numerical columns
print('Minimum value: ')
slimmed_train_data.iloc[:,:].min()

print('Maximum value: ')
slimmed_train_data.iloc[:,:].max()


Minimum value: 
Maximum value: 


id                                             6334010
target                                               1
comment_text                           🤣gotta love it!
severe_toxicity                                      1
obscene                                              1
identity_attack                                      1
insult                                               1
threat                                               1
asian                                                1
atheist                                              1
bisexual                                             1
black                                                1
buddhist                                             1
christian                                            1
female                                               1
heterosexual                                         1
hindu                                                1
homosexual_gay_or_lesbian                            1
intellectu

what can be deleted from messages field?
-stop words, punctuation, emoji, 

lematization or stemming
output of wrangling - corpus can be part of a data frame with other cleaned up columns

- Why removing meta-data columns? There has to be a data-science driven reason ...
    1) can't delete columns from dataset without approval from owner of data
    2) just because I don't see a connection doesn't mean there is one
    3) how can use the information that we're given to create a potential feature that is a measure the level of intention - like or don't like

- Null values implies values that are missing. Is it possible to have values that are missing that are not null?
    1) cases where values are not null but can still be not good
 
- How would we check for data quality?
    1) max, min, range, content, etc.

- There seem to be many columns with about 80% of values that are missing. How do we deal with this?
    1) Could limit the data to the ~20% of the data with identifiers
    2) concentrate on comment_text and response variable initially?
    
    a) would it make sense to compute them myself?

- Do we know what columns are categorical and what columns are non-categorical? 
    1) I would say the identity labels are categorical
    2) target is the response
    3) what would comment_text be?
    non-categorical predominantly - continuous metric
    categorical - comment_text, refers to a class

- Think about ways of persisting the dataset as an output of the wrangling phase, which will be an input to the next phase storytelling). (Hint ... you can use something like Pickle, or perhaps generate a CSV file)
    1) pickle the data

emoji - encoded to a string as a token (possibility); do search on how to deal with emoji in NLP

packages - nltk, spacy

check if summation of identity labels across rows is equal to 1.0